In [1]:
import os, nltk, re, string, collections, json
from nltk.util import *
from nltk.lm import *
import xml.etree.cElementTree as et
from nltk.collocations import *
import numpy as np

In [2]:
def parseXml(folderName, wordList):
    # getting current working directory and adding folder containing dataset
    cwd = os.getcwd()
    default_cwd = cwd + folderName

    # getting list of file names in folder containing dataset
    file_names = os.listdir(os.getcwd() + folderName[:-1]) # can not use the last / here
    
    # looping every filename and checking that it ends in .xml
    for file in file_names:
        if file.endswith('.xml'):
            cwd = default_cwd + file # adding filename to file path
            tree = et.parse(cwd)
            root = tree.getroot()
            
            # looping through every sentence tag in files and adding all words and punctuation to list
            for item in root.iter('s'):
                wordList.append(startingTag)
                for word in item:
                    if(isinstance(word.text, str)):
                        # checked that string is of type string and removing extra whitespace
                        wordList.append(word.text.lower().strip())
                wordList.append(closingTag)
                
    return wordList

In [3]:
words = [] 
startingTag = "<s>"
closingTag = "</s>"
unkString = '<UNK>'

words = parseXml('\\training_set\\', words)

In [4]:
# method to get different ngrams by inputting list and length
def getNgrams(corpus, length):
    ngram = ngrams(corpus, length) 
    return collections.Counter(ngram)

In [5]:
# method to get word/s frequencies into a dictionary 
def getVanillaFreqs(ngram):
    freqDict = {}
    
    # converting ngrams to dictionaries for easier use
    ngramDict = dict(ngram)
    
    # creating lists of keys and values
    ngramKeyList = list(ngramDict.keys()) 
    ngramValueList = list(ngramDict.values()) 
    
    for i in range(len(ngramDict)):
        currKey = ngramKeyList[i] # getting word/s 
        currValue = ngramValueList[i] # getting number of appearences

        # calculating percentage of frequency
        freq = (currValue / len(words))

        freqDict.update({currKey:freq})
        
    return freqDict

In [6]:
vanillaUnigram = getNgrams(words, 1)
vanillaBigram = getNgrams(words, 2)
vanillaTrigram = getNgrams(words, 3)

In [7]:
vanillaUnigramFreqs = getVanillaFreqs(vanillaUnigram)
vanillaBigramFreqs = getVanillaFreqs(vanillaBigram)
vanillaTrigramFreqs = getVanillaFreqs(vanillaTrigram)

In [8]:
# method to smooth the counts of words 
def laplaceSmoothing(ngram):
    # converting ngrams to dictionaries for easier use
    ngramDict = dict(ngram)
    
    # creating lists of keys and values
    ngramKeyList = list(ngramDict.keys()) 
    ngramValueList = list(ngramDict.values()) 
    
    for i in range(len(ngramDict)):
        currKey = ngramKeyList[i] # getting word/s 
        currValue = ngramValueList[i] # getting number of appearences
        
        tempItem = {(currKey): currValue + 1}
        ngramDict.update(tempItem)
        
    return ngramDict

In [9]:
# method to get freq from dict of word/s counts
def getLaplaceFreqs(laplaceNgram):
    freqDict = {}
    
    # creating lists of keys and values
    ngramKeyList = list(laplaceNgram.keys()) 
    ngramValueList = list(laplaceNgram.values()) 

    for i in range(len(laplaceNgram)):
        currKey = ngramKeyList[i] # getting word/s 
        currValue = ngramValueList[i] # getting number of appearences

        # calculating percentage of frequency
        freq = (currValue / (len(words) + len(laplaceNgram)))

        freqDict.update({currKey:freq})
        
    return freqDict

In [10]:
# getting laplace smoothened counts
laplaceUnigram = laplaceSmoothing(vanillaUnigram)
laplaceBigram = laplaceSmoothing(vanillaBigram)
laplaceTrigram = laplaceSmoothing(vanillaTrigram)

In [11]:
# calculating laplace percentages
laplaceUnigramFreqs = getLaplaceFreqs(laplaceUnigram)
laplaceBigramFreqs = getLaplaceFreqs(laplaceBigram)
laplaceTrigramFreqs = getLaplaceFreqs(laplaceTrigram)

In [12]:
# method to insert <UNK> tokens in counts
def unkTokenInsertion(ngram):
    unkTreshold = 2
    toDeleteKeys = []
    # converting ngrams to dictionaries for easier use
    ngramDict = dict(ngram)
    
    # creating lists of keys and values
    ngramKeyList = list(ngramDict.keys()) 
    ngramValueList = list(ngramDict.values()) 
    
    for i in range(len(ngramDict)):
        currKey = ngramKeyList[i] # getting word/s 
        currValue = ngramValueList[i] # getting number of appearences
        
        if currValue <= unkTreshold:
            # create UNK dictionary entry if not present
            if ngramDict.get(unkString) == None:
                ngramDict.update({unkString:1})
            else:
                unkValue = ngramDict.get(unkString) + 1
                ngramDict.update({unkString:unkValue})
                # adding key to list of keys to delete
                toDeleteKeys.append(currKey)
        else:
            tempItem = {(currKey): currValue + 1}
            ngramDict.update(tempItem)
    
    # deleting items after to avoid out of bounds exception
    for i in range(len(toDeleteKeys)):
        del ngramDict[toDeleteKeys[i]]
        
    return ngramDict

In [13]:
def getUnkFreqs(unkNgram):
    freqDict = {}
    
    # creating lists of keys and values
    ngramKeyList = list(unkNgram.keys()) 
    ngramValueList = list(unkNgram.values()) 
    
    for i in range(len(unkNgram)):
        currKey = ngramKeyList[i] # getting word/s 
        currValue = ngramValueList[i] # getting number of appearences

        # calculating percentage of frequency
        freq = (currValue / (len(words) + len(unkNgram)))

        freqDict.update({currKey:freq})
        
    return freqDict

In [14]:
# getting word/s counts with <UNK> token inserted
unkUnigram = unkTokenInsertion(vanillaUnigram)
unkBigram = unkTokenInsertion(vanillaBigram)
unkTrigram = unkTokenInsertion(vanillaTrigram)

In [15]:
# calculating unk model percentages
unkUnigramFreqs = getUnkFreqs(unkUnigram)
unkBigramFreqs = getUnkFreqs(unkBigram)
unkTrigramFreqs = getUnkFreqs(unkTrigram)

In [16]:
# adding starting (<s>) and closing (</s>) tags to list of words for computation
def addSTags(wordList):
    wordList.insert(0, startingTag)
    wordList.append(closingTag)
    return wordList

In [17]:
# calculations related to probability to reduce code redundancy
def probabilityCalc(model, wordList, ngramLen, unigramFreq, unigram, bigram, trigram):
    res = 1
    
    if model == 1:
        if ngramLen == 1:
            freqList = []
            
            for i in range(len(wordList)):
                # checking if all words exist
                if unigram[(wordList[i],)] == 0:
                    return 0
                
                # getting freq of every word and storing them in seperate list
                freqList.append(unigramFreq[(wordList[i],)])
            for i in range(len(freqList)):
                res *= freqList[i]

            return res
        elif ngramLen == 2:
            wordList = addSTags(wordList)

            for i in range(len(wordList) - 1):
                if bigram[(wordList[i], wordList[i+1])] == 0:
                    return 0
                
                if unigram[(wordList[i],)] == 0:
                    lastWord = 1
                else:
                    lastWord = unigram[(wordList[i],)]
                prob = bigram[(wordList[i], wordList[i+1])] / lastWord
                res *= prob

            return res
        elif ngramLen == 3:
            wordList = addSTags(wordList)
            
            for i in range(len(wordList) - 2):
                if trigram[(wordList[i], wordList[i+1], wordList[i+2])] == 0:
                    return 0
                # avoiding division by 0 
                if bigram[(wordList[i], wordList[i+1])] == 0:
                    lastTwoWords = 1
                else:
                    lastTwoWords = bigram[(wordList[i], wordList[i+1])]
                prob = trigram[(wordList[i], wordList[i+1], wordList[i+2])] / lastTwoWords
                res *= prob
            
            return res
        else:
            print("Not valid n-gram count.")
    elif model == 2:        
        if ngramLen == 1:           
            freqList = []
            
            for i in range(len(wordList)):
                try:
                    unigram[wordList[i],]
                except Exception:
                    return 0
                # getting freq of every word and storing them in seperate list
                freqList.append(unigramFreq[(wordList[i],)])
            for i in range(len(freqList)):
                res *= freqList[i]

            return res
        elif ngramLen == 2:
            wordList = addSTags(wordList)

            for i in range(len(wordList) - 1):
                try:
                    bigram[wordList[i], wordList[i+1]]
                except Exception:
                    return 0
                prob = bigram[(wordList[i], wordList[i+1])] / unigram[(wordList[i],)]
                res *= prob

            return res
        elif ngramLen == 3:
            wordList = addSTags(wordList)
            
            for i in range(len(wordList) - 2):
                try:
                    trigram[wordList[i], wordList[i+1], wordList[i+2]]
                except Exception:
                    return 0
                
                if bigram[(wordList[i], wordList[i+1])] == 0:
                    lastTwoWords = 1
                else:
                    lastTwoWords = bigram[(wordList[i], wordList[i+1])]
                    prob = trigram[(wordList[i], wordList[i+1], wordList[i+2])] / lastTwoWords
                    res *= prob
            
            return res
        else:
            print("Not valid n-gram count.")
    # extra bits for unk calculations                                   
    elif model == 3:
        if ngramLen == 1:
            freqList = []
            # getting freq of every word and storing them in seperate list
            for i in range(len(wordList)):
                try:
                    currWordFreq = unigramFreq[(wordList[i],)]
                except Exception:
                    currWordFreq = unigramFreq[unkString]
                freqList.append(currWordFreq)
                
            for i in range(len(freqList)):
                res *= freqList[i]

            return res
        elif ngramLen == 2:
            wordList = addSTags(wordList)
             
            for i in range(len(wordList) - 1):
                # if phrases are not in vocab then replace count by UNK tag
                try:
                    allPhrase = bigram[(wordList[i], wordList[i+1])]
                except Exception:
                    allPhrase = bigram[unkString]
                try:
                    firstWord = unigram[(wordList[i],)]
                except Exception:
                    firstWord = unigram[unkString]
                
                prob = firstWord / allPhrase
                res *= prob
                
            return res
        elif ngramLen == 3:
            wordList = addSTags(wordList)
             
            for i in range(len(wordList) - 2):
                # if phrases are not in vocab then replace count by UNK tag
                try:
                    allPhrase = trigram[(wordList[i], wordList[i+1], wordList[i+2])]
                except Exception:
                    allPhrase = trigram[unkString]
                try:
                    firstTwoWords = bigram[(wordList[i], wordList[i+1])]
                except Exception:
                    firstTwoWords = bigram[unkString]
                
                prob = firstTwoWords / allPhrase 
                res *= prob
                
            return res
        else:
            print("Not valid n-gram count.")

In [18]:
# wrapper method to find probabilty of phrase (for model: 1 - Vanilla, 2 - Laplace, 3 - UNK)
def probability(string, model, ngramLen):
    # splitting the string from the spaces to get a list with the individual words
    string = string.lower()
    wordList = re.findall(r"[\w']+|[.,!?;]", string)
    
    if model == 1:
        return probabilityCalc(model, wordList, ngramLen, vanillaUnigramFreqs, vanillaUnigram, vanillaBigram, vanillaTrigram)
    elif model == 2:
        return probabilityCalc(model, wordList, ngramLen, laplaceUnigramFreqs, laplaceUnigram, laplaceBigram, laplaceTrigram)
    elif model == 3:
        return probabilityCalc(model, wordList, ngramLen, unkUnigramFreqs, unkUnigram, unkBigram, unkTrigram)
    else:
        print("Not valid model choice.")

In [19]:
# hard coded weights for ngrams
unigramLamba = 0.1
bigramLamba = 0.3
trigramLamba = 0.6

# linear interpolation method (for model: 1 - Vanilla, 2 - Laplace, 3 - UNK)
def li(string, model):
    unigramWeight = probability(string, model, 1) * unigramLamba
    bigramWeight = probability(string, model, 2) * bigramLamba
    trigramWeight = probability(string, model, 3) * trigramLamba
    
    return unigramWeight + bigramWeight + trigramWeight

In [20]:
# method to get perplexity of a sentence (li - boolean if linear interpolation should be used)
def perplexityForString(string, model, ngram, isLi):
    n = len(re.findall(r"[\w']+|[.,!?;]", string))
    if isLi == True:    
        prob = li(string, model)
        if prob == 0:
            return 0
        else:
            return pow(prob, -1/n)
    else:
        prob = probability(string, model, ngram)
        if prob == 0:
            return 0
        else:
            return pow(prob, -1/n)

In [31]:
print(perplexityForString("This is one", 3, 3, False))

2.9833589960533025


In [22]:
def perplexityForFiles(folderName, model, ngram, isLi):
    words = []
    words = parseXml(folderName, words)

    # getting sentences from words in tokenised form
    formattedWords = tempSentence = []
    counter = 0
    for i in range(len(words)):
        # if start of sentence then continue
        if words[i] == '<s>':
            continue
        elif words[i] == '</s>': 
            # if end of sentence then add completed sentence to list
            formattedWords.insert(counter, tempSentence)
            counter += 1
            tempSentence = []                   
        else:
            tempSentence.append(words[i])
    
    # first element and last 23 was being false info
    del formattedWords[0]
    del formattedWords[-23:]
    
    # taking each sentence and making it 1 string to then pass through other methods
    tempString = ""
    formattedSentences = []
    for sentence in formattedWords:
        for word in sentence:
            tempString += " " + word
        formattedSentences.append(tempString)
        tempString = ""
    
    if isLi == True:  
        totalProb = tempProb = 0 
        for i in range(len(formattedSentences)):
            tempProb = li(formattedSentences[i], model)
            totalProb += tempProb
        averageProb = totalProb / len(formattedSentences)
        
        if averageProb == 0:
            return 0
        else:
            return pow(averageProb, -1/len(words)) 
    else:
        totalProb = tempProb = 0 
        for i in range(len(formattedSentences)):
            tempProb = probability(formattedSentences[i], model, ngram)
            totalProb += tempProb
        averageProb = totalProb / len(formattedSentences)

        if averageProb == 0:
            return 0
        else:
            return pow(averageProb, -1/len(words))



In [43]:
print(perplexityForFiles('\\testing_set\\', 3, 3, True))

0.998134694600608


In [24]:
def textGenWrapper(string, model):
    if model == 1:
        return textGeneration(string, vanillaTrigramFreqs)
    elif model == 2:
        return textGeneration(string, laplaceTrigramFreqs)
    elif model == 3:
        return textGeneration(string, unkTrigramFreqs)
    else:
        print("Not a valid model selection.")

In [25]:
def textGeneration(string, model):
    finalString = string
    tokenisedString = re.findall(r"[\w']+|[.,!?;]", string)
    
    modelList = list(model.keys())
    
    lastTwo = tokenisedString[-2:]
    
    # getting tuples that include the last 2 words of inputted string
    includesTuple = [item for item in model if item[0] == lastTwo[0] and item[1] == lastTwo[1]]
    
    maxIndex = maxFreq = 0
    maxTuple = ()
    for i in range(len(includesTuple)):
        if model[includesTuple[i]] > maxFreq:
            maxFreq = model[includesTuple[i]]
            maxIndex = i
            maxTuple = includesTuple[i]
    
    # if there are no possible continuations just return
    if maxTuple == ():
        return finalString
    
    finalString += " " + maxTuple[2]
    lastTwo[0] = lastTwo[1]
    lastTwo[1] = maxTuple[2]
    
    while lastTwo[1] != '</s>':
        includesTuple = [item for item in model if item[0] == lastTwo[0] and item[1] == lastTwo[1]]
        
        maxIndex = maxFreq = 0
        maxTuple = ()
        for i in range(len(includesTuple)):
            if model[includesTuple[i]] > maxFreq:
                maxFreq = model[includesTuple[i]]
                maxIndex = i
                maxTuple = includesTuple[i]
        
        # if there are no possible continuations just return
        if maxTuple == ():
            return finalString
        
        finalString += " " + maxTuple[2]
        lastTwo[0] = lastTwo[1]
        lastTwo[1] = maxTuple[2]
    
    # removes the ending string tag
    return finalString[:-4]

In [29]:
textGenWrapper("only recently we have", 1)

"only recently we have n't got a lot of people who are you ? "

In [27]:
def exportModelToJson(model, filename):
    newDict = {}
    
    # have to convert keys from tuples to strings to be exported
    keyList = list(model.keys())
    valueList = list(model.values())
    
    for i in range(len(keyList)):
        newDict.update({str(keyList[i]):valueList[i]})
    
    with open(filename + '.json', 'w') as fp:
        json.dump(newDict, fp, indent=4)

In [28]:
# exporting all models to JSON
exportModelToJson(vanillaUnigramFreqs, "vanillaUnigramModel")
exportModelToJson(vanillaBigramFreqs, "vanillaBigramModel")
exportModelToJson(vanillaTrigramFreqs, "vanillaTrigramModel")

exportModelToJson(laplaceUnigramFreqs, "laplaceUnigramModel")
exportModelToJson(laplaceBigramFreqs, "laplaceBigramModel")
exportModelToJson(laplaceTrigramFreqs, "laplaceTrigramModel")

exportModelToJson(unkUnigramFreqs, "unkUnigamModel")
exportModelToJson(unkBigramFreqs, "unkBigramModel")
exportModelToJson(unkTrigramFreqs, "unkTrigramModel")